In [1]:
import sys
sys.path.append('/kaggle/input/packagist')
print(sys.path)

['/kaggle/working', '/kaggle/lib/kagglegym', '/kaggle/lib', '/kaggle/input/ubiquant-market-prediction', '/opt/conda/lib/python37.zip', '/opt/conda/lib/python3.7', '/opt/conda/lib/python3.7/lib-dynload', '', '/root/.local/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages', '/src/bq-helper', '/opt/conda/lib/python3.7/site-packages/IPython/extensions', '/root/.ipython', '/kaggle/input/packagist']


In [2]:
import copy
import numpy as np
import pandas as pd

from pytorch_forecasting import TemporalFusionTransformer

In [3]:
tuft = TemporalFusionTransformer.load_from_checkpoint('/kaggle/input/checkpoint0/best.ckpt')

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [4]:
import ubiquant # host's lib
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [5]:
i = -1
tester = pd.DataFrame()
bester = pd.DataFrame()

In [6]:
for (test_df, sample_prediction_df) in iter_test:
    #print(sample_prediction_df)
    #print(type(sample_prediction_df.iloc[0,1]))
    #print('\n')
    i += 1
    if 'time_id' not in test_df.columns:
        test_df['time_id'] = test_df['row_id'].apply(lambda x: int(x.split('_')[0]))
    test_df.investment_id = test_df.investment_id.astype(str)
    test_df['target'] = 0
    uni = list(test_df.investment_id.unique())
    
    if i == 0:
        tester = test_df
        initime = test_df['time_id'][0]
        # print(initime)
    else: 
        frames = [tester, test_df]
        tester = pd.concat(frames, ignore_index=True)
        tester = tester[tester['time_id'] >= initime + i - 9]
    
    bester = tester.loc[tester['investment_id'].isin(uni)].copy()
    buba = bester["investment_id"].value_counts()
    if buba.min() == 1:
        # print(buba)
        for invid in uni:
            if buba[invid] == 1:
                muma = bester[bester['investment_id'] == invid].copy()
                muma.at[muma.index[0], 'time_id'] -= 1
                dframes = [bester, muma]
                bester = pd.concat(dframes, ignore_index=True)
    
    coda = tuft.predict(bester, return_index=True)
    coda[1]['row_id'] = coda[1]['time_id'].astype(str) + '_' + coda[1]['investment_id'].astype(str)
    coda[1]['target'] = coda[0].numpy().astype(np.float64)
    pup = sample_prediction_df.merge(coda[1], how='left', on='row_id')
    
    sample_prediction_df['target'] = pup['target_y']
    #print(sample_prediction_df)
    #print(type(sample_prediction_df.iloc[0,1]))
    #print('\n')
    env.predict(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
